In [1]:
from sanic import Sanic
import uuid

import logging
import json


In [2]:
LOG_FORMAT = "%(asctime)s [%(levelname)s]: %(threadName)s - %(message)s"
logging.basicConfig(format=LOG_FORMAT)
logger = logging.getLogger(__name__)
logger.setLevel("INFO")

[x] Cada jogador inicia com 10 cartas;

[ ] ~~ Toda rodada é sorteado um jogador para receber três cartas pretas e escolher qual vai ser a carta da rodada;~~

[ ] O vencedor da rodada escolherá a carta de pergunta da rodada seguinte

[ ] ~~O jogo termina quando acabarem as cartas brancas;~~

Algumas cartas têm 2 espaços pra resposta, já está mapeado no bd;

Para cartas com 2 respostas, todos jogadores recebem +1 carta;

Regra RANDO: Toda rodada o jogador dummy recebe uma carta, caso a dele seja votada mais engraçada, o jogo acaba e todos perdem;

No fim da rodada, cada jogador vota em uma carta; A mais votada recebe um ponto; Ganha quem chegar no máximo de pontuação mais rápido;

A pontuação é definida no início da partida, ao criar a sala;

Modo sobrevivência: Após o fim das respostas, todos os jogadores votam em uma carta para ser apagada, a última restante ganha;

É possível sacrificar 1 ponto para girar suas cartas em mãos;

A cada rodada, o jogador pode puxar mais uma carta;

Cartas brancas não repetem no deck;

In [208]:
class Resposta:
    def __init__(self, texto: str, autor: str = "Cartas contra a humanidade") -> None:
        self.texto = texto
        self.ingame = False
        self.autor = autor
        self.jogador = None
        self.votos = 0
        self.id = uuid.uuid4()

    def marcar_ingame(self):
        self.ingame = True

    def to_json(self) -> str:
        obj = {"texto": self.texto, "autor": self.autor, "id": self.id}

        return json.dumps(obj)

    def debug(self):
        return f"""Texto: {self.texto}\nIngame: {self.ingame}\nVotos: {self.votos}\nID: {self.id}\nAutor: {self.autor}\nJogador: {self.jogador}"""


In [210]:

class Jogador:
    def __init__(self, nome: str) -> None:
        self.nome = nome
        self.cartas = []
        self.pontuacao = 0
        self.status = "aguardando"  # possíveis status = aguardando, escolhendo, votando
        self.id = uuid.uuid4()

        logger.info("Jogador %s foi criado com o id %s.", self.nome, self.id)

    def descartar_resposta(self, resposta: Resposta):

        logger.info(
            "Carta recebida foi de texto %s e ID %s", resposta.texto, str(resposta.id)
        )

        self.cartas = [
            carta for carta in self.cartas if str(str(carta.id)) != str(resposta.id)
        ]

        logger.info(
            "A carta de ID %s foi descartada do deck do jogador %s",
            str(resposta.id),
            self.id,
        )
        logger.info("Agora o jogador %s possui %s cartas.", self.nome, len(self.cartas))

    def to_json(self) -> str:
        response = {
            "nome": self.nome,
            "id": self.id,
        }

        return response

    def debug(self) -> None:
        print(str(self))

    def __str__(self) -> str:
        txt = f"""\n\tNome: {self.nome}\n\tCartas: {','.join([carta.texto for carta in self.cartas])}\n\tPontuação: {self.pontuacao}\n\tStatus: {self.status}\n\tID: {self.id}"""

        return txt


In [211]:
class Pergunta:
    def __init__(self, texto: str, autor: str = "default") -> None:
        self.texto = texto
        self.ingame = False
        self.picks = max(texto.count("_"), 1)
        self.autor = autor
        self.id = uuid.uuid4()

    def to_json(self) -> str:
        obj = {
            "texto": self.texto,
            "picks": self.picks,
            "autor": self.autor,
            "id": str(self.id),
        }

        return json.dumps(obj)

    def debug(self):
        return f"""Texto: {self.texto}\nIngame: {self.ingame}\nPicks: {self.picks}\nID: {self.id}\nAutor: {self.autor}"""


In [147]:
from random import choice
class Deck:
    """### Responsabilidades do Deck

    Ler as cartas da memória

    Selecionar as cartas para serem escolhidas como pergunta

    Distribuir as cartas para os jogadores

    Marcar cartas como ingame (Perguntas e Respostas)

    """

    def __init__(self) -> None:
        self.perguntas = []
        self.respostas = []

        logger.info("Deck foi iniciado.")
        self.carregar_cartas()

    def carregar_cartas(self):
        """Faz a leitura das cartas no banco de dados"""
        with open(
            "/home/costa/Projetos/cards_against_the_humanity/server/src/objects/perguntas.txt"
        ) as f:
            for pergunta in f.readlines():
                self.perguntas.append(Pergunta(pergunta[:-2]))

            logger.info("Foram carregadas %s perguntas", len(self.perguntas))

        with open(
            "/home/costa/Projetos/cards_against_the_humanity/server/src/objects/respostas.txt"
        ) as f:
            for resposta in f.readlines():
                self.respostas.append(Resposta(resposta[:-2]))
            logger.info("Foram carregadas %s respostas", len(self.respostas))

    def pick_respostas(self, k: int = 10):
        """Seleciona 10 cartas arbitrárias entre as cartas que ainda não estão em jogo.
        Automaticamente as marca como ingame.

        Args:
            k (int, optional): Quantidade de cartas a serem puxadas. Defaults to 10.

        Raises:
            Exception: Caso acabem as cartas disponíveis, levanta um erro

        Returns:
            list(Resposta): Lista com as 10 respostas já marcadas como ingame
        """
        respostas_nao_usadas = [
            resposta for resposta in self.respostas if resposta.ingame == False
        ]
        logger.info(f"respostas restantes: {len(respostas_nao_usadas) - 10}")

        if len(respostas_nao_usadas) == 0:
            raise Exception("Acabaram as cartas, porra")

        respostas = [choice(respostas_nao_usadas) for i in range(k)]

        for resposta in respostas:
            resposta.marcar_ingame()

        if k == 1:
            # Caso onde é a primeira carta a ser jogada
            # não passa pelo processo de seleção dos jogadores
            self.marcar_resposta_ingame(respostas[0].id)

        return respostas

    def pick_perguntas(self, k: int = 3) -> list[Pergunta]:
        """Escolhe aleatoriamente 3 perguntas da lista de perguntas ainda não escolhidas

        Params:
            k (int): Quantidade de perguntas a serem retornadas

        Raises:
            Exception: Erro caso não hajam mais cartas para serem escolhidas

        Returns:
            list[Pergunta]: Lista com as 3 perguntas para o jogador escolher.
        """
        perguntas_nao_usadas = [
            pergunta for pergunta in self.perguntas if pergunta.ingame == False
        ]

        logger.info(f"Perguntas restantes: {len(perguntas_nao_usadas)}")

        if len(perguntas_nao_usadas) == 0:
            raise Exception("Acabaram as cartas, porra")

        perguntas = [choice(perguntas_nao_usadas) for i in range(k)]

        if k == 1:
            # Caso onde é a primeira carta a ser jogada
            # não passa pelo processo de seleção dos jogadores
            self.marcar_pergunta_ingame(perguntas[0].id)

        return perguntas

    def marcar_pergunta_ingame(self, id):
        """Recebe um ID de pergunta e marca ela como já utilizada em jogo

        Args:
            id (str): identificador da carta a ser marcada

        Raises:
            Exception: Exception de not found
        """
        pergunta = [pergunta for pergunta in self.perguntas if pergunta.id == id].pop()
        if not pergunta:
            raise Exception("Pergunta não existe na lista de perguntas")
        pergunta.ingame = True

        logger.info("%s", pergunta.debug())


# Passo a passo da partida

[x] Criar a partida com os parâmetros de jogo

[x] Esperar todos os jogadores entrarem

[x] Iniciar o jogo (apenas o dono)

[x] Distribuir as cartas para cada jogador

[x] Selecionar a carga de pergunta da rodada (apenas a primeira rodada)

[ ] Cada jogador recebe sua carta

[ ] Cada jogador pode puxar uma carta nova 1x por rodada

[ ] Votação

[ ] Vencedor escolhe a carta pergunta das próximas rodadas

[ ] Repete o jogo em ciclo até a pontuação máxima chegar ou o máximo de rodadas

In [212]:
class Mesa:

    def __init__(self, deck: Deck) -> None:
        self.votos = 0
        self.deck = deck

        self.pergunta_da_rodada = None
        self.respostas_da_rodada = []
        logger.info("Mesa foi iniciada.")

    def reiniciar(self):
        self.pergunta_da_rodada = []
        self.respostas_da_rodada = []

    def adicionar_resposta(self, resposta: Resposta):
        logger.info(
            "A resposta '%s' foi adicionada à mesa e marcada como usada ingame",
            resposta.texto,
        )
        self.respostas_da_rodada.append(resposta)

    def computar_voto(self, id_resposta):
        obj = [
            resposta
            for resposta in self.respostas_da_rodada
            if str(resposta.id) == id_resposta
        ]

        if len(obj) > 0:
            obj[0].votos += 1
            self.votos += 1
        else:
            raise Exception("Carta não encontrada")

        logger.info("A respost '%s' agora possui %s votos.", obj[0].texto, obj[0].votos)

    def finalizar_rodada(self) -> list[Resposta]:
        def __return_votos(resposta: Resposta):
            return resposta.votos

        votados = sorted(self.respostas_da_rodada, key=__return_votos)
        return votados

    def debug(self):
        return f"Pergunta da rodada: \n\tTexto: {self.pergunta_da_rodada.texto}\n\tPicks: {self.pergunta_da_rodada.picks}\nRespostas em jogo: {[resposta.texto for resposta in self.respostas_da_rodada]}"


In [197]:
class Partida:
    """
    Entidade da partida
    """
    def __init__(
        self,
        jogador: Jogador,
        max_pontuacao: int = 10,
        temporizador_votacao: int = 30,
        temporizador_selecao: int = 30,
        max_rodadas: int = 15,
    ) -> None:

        # Valores devault
        self.temporizador_votacao = temporizador_votacao
        self.temporizador_selecao = temporizador_selecao
        self.max_pontuacao = max_pontuacao
        self.max_rodadas = max_rodadas
        self.dono = jogador

        self.jogadores = [self.dono]
        self.rodada_atual = 0
        self.deck = Deck()
        self.mesa = Mesa(deck=self.deck)
        self.id = uuid.uuid4()
        self.started = False

        logger.info('A entidade da Partida foi criada com o ID %s', self.id)

    def __str__(self) -> str:
        debug_text = f"""ID: {self.id}\nStarted: {self.started}\nTemporizador de seleção: {self.temporizador_selecao}\nTemporizador de votação: {self.temporizador_votacao}\nPontuação máxima: {self.max_pontuacao}\nMáximo de rodadas: {self.max_rodadas}\nJogadores: {len(self.jogadores)}\nDono da sala: {str(self.dono)}\n"""
        return debug_text

    def iniciar_jogo(self, id: str) -> str:
        try:
            if id == str(self.dono.id):
                self.started = True

                for jogador in self.jogadores:
                    jogador.cartas.extend(self.deck.pick_respostas())
                    
                    for carta in jogador.cartas:
                        carta.jogador = str(jogador.id)

                payload = self.iniciar_rodada()

                return 'OK'
            else:
                return 'NOT STARTED'
            
        except Exception as e:
            logger.exception(e)

    def iniciar_rodada(self, id: str = None):
        if id == None:
            pergunta = self.gerar_primeira_pergunta()
            payload = pergunta.to_json()
        
        else:
            perguntas = self.deck.pick_perguntas(3)
            payload = [pergunta.to_json() for pergunta in perguntas]

        return payload

    def gerar_primeira_pergunta(self) -> None:
        pergunta_inicial = self.deck.pick_perguntas(1)
        self.mesa.pergunta_da_rodada = pergunta_inicial[0]

        return pergunta_inicial[0]

    def gerar_perguntas(self):
        perguntas = self.deck.pick_perguntas()
        return perguntas

    def iniciar_vocatacao(self):
        raise NotImplementedError
    
    def finalizar_votacao(self):

        ranking = self.mesa.finalizar_rodada()
        vencedor = ranking[0].jogador

        for jogador in self.jogadores:
            if str(jogador.id) == vencedor:
                jogador.pontuacao += 1
                logger.info("O jogador de id '%s' foi o vencedor da rodada e agora possui %s pontos.", vencedor, jogador.pontuacao)

    def contabilizar_voto(self, id_resposta):
        try:
            self.mesa.computar_voto(id_resposta)
        except Exception as e:
            logger.info(e)

        if self.mesa.votos == len(self.jogadores):
            self.finalizar_votacao()

    def selecionar_pergunta(self, payload: dict):

        jogador = self.get_jogador(payload['jogador'])
        
        print(f'jogador: {jogador.nome}')

        if len(payload['resposta']) > int(self.mesa.pergunta_da_rodada.picks):
            logger.info("O jogador %s enviou mais respostas do que o necessário.", jogador.nome)
            raise Exception("Mais respostas do que deveria.")

        ja_jogaram = [str(resposta.jogador) for resposta in self.mesa.respostas_da_rodada]

        if jogador.nome in ja_jogaram:
            raise Exception("Jogador já fez sua seleção.")
        
        else:
            # logger.info("Jogadores que já responderam a rodada: %s", ja_jogaram)

            for resposta in payload['resposta']:
                resposta = self.get_resposta(resposta)

                try:
                    self.mesa.adicionar_resposta(resposta)
                    jogador.descartar_resposta(resposta)
                except Exception as e:
                    logger.exception(e)
                    raise Exception(e)

    def entrar_na_sala(self, novo_jogador: Jogador) -> uuid.UUID:
        try:
            if novo_jogador.id not in [jogador.id for jogador in self.jogadores]:
                self.jogadores.append(novo_jogador)
            else:
                raise Exception('Jogador de mesmo ID já está no lobby')
            
            logger.info("O jogador %s entrou na sala com o ID %s", novo_jogador.nome, novo_jogador.id)
            return self.id
        except Exception as e:
            logger.exception(e)
        
    def get_resposta(self, id_resposta: str):
        resposta = [resposta for resposta in self.deck.respostas if str(resposta.id) == id_resposta]
        if len(resposta) > 0:
            return resposta[0]

    def get_jogador(self, id_jogador: str):
        jogador = [jogador for jogador in self.jogadores if str(jogador.id) == id_jogador]
        
        if len(jogador) > 0:
            return jogador[0]

    def debug_jogadores(self) -> None:
        for jogador in self.jogadores:
            print(str(jogador))

    def debug(self) -> None:
        print(str(self))


In [198]:
dummy = Jogador('isaac')
dummy_2 = Jogador('vitu')

2024-04-28 23:37:49,595 [INFO]: MainThread - Jogador isaac foi criado com o id dcf74ead-ea71-4683-a257-0a8a518e85cb.
2024-04-28 23:37:49,596 [INFO]: MainThread - Jogador vitu foi criado com o id 1dce7492-9b8a-4591-a444-884b9665dfb5.


In [199]:
game = Partida(dummy, 10, 30, 15)

game.entrar_na_sala(dummy_2)

2024-04-28 23:37:49,742 [INFO]: MainThread - Deck foi iniciado.
2024-04-28 23:37:49,743 [INFO]: MainThread - Foram carregadas 95 perguntas
2024-04-28 23:37:49,746 [INFO]: MainThread - Foram carregadas 403 respostas
2024-04-28 23:37:49,746 [INFO]: MainThread - Mesa foi iniciada.
2024-04-28 23:37:49,746 [INFO]: MainThread - A entidade da Partida foi criada com o ID 43241c0e-5f89-4ddf-afa0-8d618405ca64
2024-04-28 23:37:49,747 [INFO]: MainThread - O jogador vitu entrou na sala com o ID 1dce7492-9b8a-4591-a444-884b9665dfb5


UUID('43241c0e-5f89-4ddf-afa0-8d618405ca64')

In [200]:
game.iniciar_jogo(str(dummy.id))

2024-04-28 23:37:49,897 [INFO]: MainThread - respostas restantes: 393
2024-04-28 23:37:49,898 [INFO]: MainThread - respostas restantes: 383
2024-04-28 23:37:49,899 [INFO]: MainThread - Perguntas restantes: 95
2024-04-28 23:37:49,899 [INFO]: MainThread - Texto: A excursão da escola foi um fiasco porque _.
Ingame: True
Picks: 1
ID: de506f13-beca-44a3-acdb-1eed49f06dd3
Autor: default


'OK'

In [201]:
game.debug_jogadores()


	Nome: isaac
	Cartas: Atendente de Telemarketing,Complexo de Édipo,Fazer um cafuné,Coceira no toba,Jogador de LOL (League of Legends),Barney (How I Met Your Mother),Pôneis malditos,Sem-vergonhice,A fome,Comer todos os doces que seriam doados no Cosme e Damião
	Pontuação: 0
	Status: aguardando
	ID: dcf74ead-ea71-4683-a257-0a8a518e85cb

	Nome: vitu
	Cartas: Os nazistas,O Projeto Tamar,Obesidade,A Paz Mundial,Passar o rodo,Gravidez na adolescência,Freud explica,Sofrimento e desespero,Ejaculação precoce,Humor negro
	Pontuação: 0
	Status: aguardando
	ID: 1dce7492-9b8a-4591-a444-884b9665dfb5


In [202]:
# Simulação de carta jogada

payload = {
  'jogador': str(dummy_2.id),
  'resposta': [str(dummy_2.cartas[0].id)]
}

In [203]:
game.selecionar_pergunta(payload=payload)

2024-04-28 23:37:50,286 [INFO]: MainThread - A resposta 'Os nazistas' foi adicionada à mesa e marcada como usada ingame
2024-04-28 23:37:50,287 [INFO]: MainThread - Carta recebida foi de texto Os nazistas e ID bc68b0d1-6e53-4697-a6ed-5b7c22b1d3f1
2024-04-28 23:37:50,288 [INFO]: MainThread - A carta de ID bc68b0d1-6e53-4697-a6ed-5b7c22b1d3f1 foi descartada do deck do jogador 1dce7492-9b8a-4591-a444-884b9665dfb5
2024-04-28 23:37:50,288 [INFO]: MainThread - Agora o jogador vitu possui 9 cartas.


jogador: vitu
[UUID('eb1bf57d-044c-4ca0-b056-34e0bb63e69f'),
 UUID('a3046c42-d5ff-430d-a2f7-67906f209348'),
 UUID('2ca560d4-9410-4e7d-b2c0-d0e079e7f171'),
 UUID('6ac036a1-3f49-482e-b96e-d66328e2f668'),
 UUID('0e65ca68-4705-4c07-9811-da4b3848b191'),
 UUID('7f067776-b326-4357-aff6-06a2a72ef5a9'),
 UUID('83d30d27-bb2b-4e16-885d-9df0231ecfb5'),
 UUID('cee2c44c-9748-4aa0-8a33-8eb46ae5b23b'),
 UUID('7acb9a8b-40c2-4d7c-8e99-018715ac9367')]


In [204]:
game.debug_jogadores()


	Nome: isaac
	Cartas: Atendente de Telemarketing,Complexo de Édipo,Fazer um cafuné,Coceira no toba,Jogador de LOL (League of Legends),Barney (How I Met Your Mother),Pôneis malditos,Sem-vergonhice,A fome,Comer todos os doces que seriam doados no Cosme e Damião
	Pontuação: 0
	Status: aguardando
	ID: dcf74ead-ea71-4683-a257-0a8a518e85cb

	Nome: vitu
	Cartas: O Projeto Tamar,Obesidade,A Paz Mundial,Passar o rodo,Gravidez na adolescência,Freud explica,Sofrimento e desespero,Ejaculação precoce,Humor negro
	Pontuação: 0
	Status: aguardando
	ID: 1dce7492-9b8a-4591-a444-884b9665dfb5


In [205]:
print(game.mesa.debug())


Pergunta da rodada: 
	Texto: A excursão da escola foi um fiasco porque _.
	Picks: 1
Respostas em jogo: ['Os nazistas']


In [206]:
game.contabilizar_voto(str(game.mesa.respostas_da_rodada[0].id))

2024-04-28 23:37:50,834 [INFO]: MainThread - A respost 'Os nazistas' agora possui 1 votos.


In [207]:
game.finalizar_votacao()

2024-04-28 23:37:51,160 [INFO]: MainThread - O jogador de id '1dce7492-9b8a-4591-a444-884b9665dfb5' foi o vencedor da rodada e agora possui 1 pontos.
